In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

# Load election data
data_dir = "../data/"
election_data = pd.read_csv(data_dir + "G20192-Columnes-ME-EN.csv", sep=';')

# Load GeoJSON file
map_catalonia='municipality_map.geojson'
geo_data = gpd.read_file(data_dir + map_catalonia)
geo_data

In [ ]:
# Translate column used to merge datasets based on this
geo_data.rename(columns={'nom_muni': 'Municipality_name'}, inplace=True)
geo_data.rename(columns={'codiine': 'Municipality_code'}, inplace=True)
# Fix problem with missing 0 at the beginning of postcode
election_data['Municipality_code'] = election_data['Municipality_code'].astype(str).apply(lambda x: '0' + x if x.isdigit() and len(x) == 4 else x)

election_data['Municipality_code']

In [ ]:
# Sum up the votes for each municipality
votes_by_municipality = election_data.iloc[:, 21:].groupby(by=election_data['Municipality_code']).sum()
votes_by_municipality.reset_index()

votes_by_municipality


In [ ]:
# Find the winner party for each municipality
winner_by_municipality=votes_by_municipality.idxmax(axis=1)
winner_by_municipality.reset_index()
list_winners=list(winner_by_municipality)
# Add column with winner party
votes_by_municipality['winner']=list_winners
# Sort by municipality
votes_by_municipality.sort_values(by='Municipality_code')
votes_by_municipality

In [ ]:
# Merge municipalities map and votes by municipality
merged_data = geo_data.merge(votes_by_municipality, on='Municipality_code')
#merged_data.to_csv('election_results_municipality.csv', index=False)

In [ ]:
# Plot the election results on a map of Catalonian municipalities
fig = px.choropleth_mapbox(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color='winner',  # Change to the column you want to visualize
    hover_name='Municipality_code',
    mapbox_style="carto-positron",
    center={"lat": 41.8781, "lon": 1.7834},  # Center of Catalonia
    zoom=7,
    color_discrete_sequence=["goldenrod",  "lightgreen", "black", "darkred", "royalblue",'purple'],
    title='Election results by municipality'
)
fig.update_layout(height=600, width=800)
fig.update_geos(fitbounds="locations", visible=False)

fig.show()